In [0]:
https://docs.databricks.com/aws/en/database-objects

In [0]:
drop schema bronze;

In [0]:
create schema if not exists dataengineer.bronze;

In [0]:
create table dataengineer.bronze.test(id int, name string, age int);
insert into dataengineer.bronze.test values(1, 'Kaudi', 36)

In [0]:
select * from dataengineer.bronze.test

In [0]:
use catalog dataengineer;
use schema bronze;
create or replace table customers_reviews(
  customer_id int,
  first_name string,
  last_name string,
  review string 
);
insert into customers_reviews values
(1, 'John', 'Doe', 'Amazing product! I love it.'),
(2, 'Jane', 'Smith', 'Not good, I am disappointed.'),
(3, 'Alice', 'Brown', 'Decent quality but can be better.'),
(4, 'Bob', 'Johnson', 'Fantastic service! Highly recommend.'),
(5, 'Charlie', 'Davis', 'Terrible experience, never buying again.');

In [0]:
select * from dataengineer.bronze.customers_reviews;

In [0]:
create function if not exists dataengineer.bronze.full_name_udf(first_name string, last_name string)
returns string
return concat(first_name, ' ', last_name)

In [0]:
select customer_id, dataengineer.bronze.full_name_udf(first_name, last_name) as full_name, review from dataengineer.bronze.customers_reviews

In [0]:
-- CREATE a complex UDF for sentiment analysis--

create function if not exists dataengineer.bronze.sentiment_analysis(review string)
returns string
return
  case
    when review like '%amazing%' or review like '%love%' or review like '%fantastic%' or review like '%recommend%' then 'postive'
    when review like '%not good%' or review like '%disappointed%' or review like '%terrible%' or review like '%never buying%' then 'negative'
    else 'neutral'
  end;


In [0]:
DROP FUNCTION IF EXISTS dataengineer.bronze.analysis;

In [0]:
select customer_id, review, dataengineer.bronze.sentiment_analysis(review) as sentiment from 
dataengineer.bronze.customers_reviews;

In [0]:
create schema if not exists dataengineer.silver;
create schema if not exists dataengineer.gold;

In [0]:

create or replace view dataengineer.silver.customers_sentiment as
(select customer_id, review, dataengineer.bronze.sentiment_analysis(review) as sentiment from 
dataengineer.bronze.customers_reviews)
 

In [0]:
select * from dataengineer.silver.customers_sentiment

In [0]:
create or replace view dataengineer.gold.customers_sentiment as
  select
      dataengineer.bronze.sentiment_analysis(review) as sentiment,
      count(*) as sentiment_count
   from 
dataengineer.bronze.customers_reviews 
group by 
  sentiment

In [0]:
select * from dataengineer.gold.customers_sentiment

Databricks visualization. Run in Databricks to view.